In [3]:
import pandas as pd

In [4]:
## mapping

mat_codes = {
    "F": "01",
    "G": "02", 
    "H": "03", 
    "J": "04", 
    "K": "05",
    "M": "06",
    "N": "07", 
    "Q": "08",
    "U" : "09",
    "V": "10",
    "X": "11",
    "Z": "12"
}

## 1. Data Preparation

In [149]:
df = pd.read_csv('commodities.csv')
df['datetime'] = pd.to_datetime(df['date'], utc=True)
df['date_strf']=df.datetime.dt.strftime('%Y%m%d')
df['dayofweek'] = df['datetime'].dt.strftime("%a")
df['month'] = df['datetime'].dt.month

df['datetime_maturity'] = pd.to_datetime(df.maturity.str[-4:] + df.maturity.str[0].map(mat_codes), format='%Y%m', utc=True)

df['datetime_maturity'] = df['datetime_maturity']-pd.Timedelta(1, "d")
df['date_strf_maturity']=df.datetime_maturity.dt.strftime('%Y%m%d')
df['time2maturity_d'] = (df.datetime_maturity-df.datetime).dt.days

df.loc[df['time2maturity_d'] <0, 'time2maturity_d'] = 0

df['time2maturity_m'] = (df.time2maturity_d/30).round()
df_settle = df.loc[df['observation'] == 'Settle']
df_soy = df_settle.loc[df_settle['instrument'] == 'CBOT.ZS']
df_soy.sort_values(['datetime' , 'datetime_maturity'], ascending = [True, False], inplace=True)




In [150]:
## select prices with only 6 months maturity for comparability

dates = set(df_soy['date_strf'])

concat = []

for i in dates:
    duration = 6
    data = df_soy.loc[(df_soy['date_strf'] == i) & (df_soy['time2maturity_m'] == duration)]
    while data.shape[0] ==0:
        duration +=1
        data = df_soy.loc[(df_soy['date_strf'] == i) & (df_soy['time2maturity_m'] == duration)]
    concat.append(data)

df_soy_6m = pd.concat(concat)
df_soy_6m.set_index('date_strf' , inplace=True, drop=True)

### Create Features

In [151]:
## calculate returns for the previous 7 days

for i in range(7):
    df_soy_6m[f'pct_t-{i+1}'] = df_soy_6m.value.pct_change(i+1)

## drop missing values
df_soy_6m.dropna(inplace=True)

In [266]:
## rolling averages for prices and returns

vals = ['value', 'pct_t-1']

for i in [7, 15, 30 , 60]:
    for j in vals:
        df_soy_6m[f'roll_avg_pct_{i}'] = df_soy_6m[j].rolling(i).mean()


for i in vals:
    df_soy_6m[f'exp_avg_{i}'] = df_soy_6m[i].expanding(1).mean()


In [267]:
df_soy_6m['pct_t-1'].rolling(7)

Rolling [window=7,center=False,axis=0,method=single]

In [268]:
desc = df_soy_6m.describe()
desc

,value,month,time2maturity_d,time2maturity_m,pct_t-1,pct_t-2,pct_t-3,pct_t-4,pct_t-5,pct_t-6,pct_t-7,roll_avg_pct_7,roll_avg_pct_15,roll_avg_pct_30,roll_avg_pct_60,exp_avg_value,exp_avg_pct_t-1
count,224.000000,224.000000,224.000000,224.000000,224.000000,224.000000,224.000000,224.000000,224.000000,224.000000,224.000000,218.000000,210.000000,195.000000,165.000000,224.000000,224.000000
mean,1313.830357,6.129464,191.678571,6.406250,0.001825,0.001956,0.002542,0.002645,0.002796,0.002845,0.002802,0.001811,0.001794,0.001633,0.001566,1314.045420,0.001658
std,54.810740,3.161743,17.775992,0.518843,0.059841,0.063921,0.059336,0.056485,0.054814,0.061519,0.058298,0.008372,0.004181,0.001944,0.001002,4.308176,0.002026
min,1204.500000,1.000000,165.000000,6.000000,-0.155849,-0.161020,-0.130147,-0.133471,-0.141960,-0.141090,-0.128337,-0.018145,-0.010578,-0.003252,-0.000716,1291.275000,-0.015441
25%,1272.500000,3.000000,176.000000,6.000000,-0.039288,-0.039682,-0.037782,-0.035065,-0.031091,-0.041387,-0.038296,-0.003790,-0.001479,0.000288,0.000876,1312.107894,0.001382
50%,1303.250000,6.000000,190.000000,6.000000,-0.000681,0.000658,-0.001622,0.003461,0.000097,0.004931,0.005260,0.002320,0.002001,0.001616,0.001493,1313.173872,0.001589
75%,1347.875000,9.000000,207.000000,7.000000,0.042275,0.041104,0.041940,0.043217,0.039182,0.045452,0.041441,0.007433,0.004376,0.003056,0.002122,1316.424178,0.001949
max,1461.000000,12.000000,225.000000,8.000000,0.144623,0.187409,0.177360,0.140865,0.189680,0.200870,0.151933,0.021951,0.012584,0.006360,0.003959,1323.478261,0.011231


In [254]:
def return_tot(df, date, value):
    period_beg = df.date.min()
    period_end = df.date.max()

    price_beg = df.loc[df.date == period_beg, value].values[0]
    price_end = df.loc[df.date == period_end, value].values[0]

    ret = (price_end-price_beg)/ price_beg
    return ret

sharpe = desc.loc['mean', 'pct_t-1'] / desc.loc['std', 'pct_t-1']
avg_return = desc.loc['mean', 'pct_t-1']
tot_return = return_tot(df_soy_6m, 'date', 'value')

print(f'Sharpe ratio: {100*(sharpe):.2f} %')
print(f'Average daily return: {100*(avg_return):.2f} %')
print(f'Total return: {100*(tot_return):.2f} %')

Sharpe ratio: 3.05 %
Average daily return: 0.18 %
Total return: -4.12 %


In [198]:
import plotly.figure_factory as ff
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

df_sorted = df_soy_6m.sort_values('date')
fig = make_subplots(rows=2, cols=2, subplot_titles=["Price", "Daily Return Distribution", 'Price Distribution', 'Return Distribution'])


dist = ff.create_distplot([df_soy_6m['pct_t-1'].values.tolist()], [''], bin_size=.01).data[1]
hist = ff.create_distplot([df_soy_6m['pct_t-1'].values.tolist()], [''], bin_size=.01).data[0]
price = px.line(x=df_soy_6m.value, y=df_soy_6m.index)
price = px.line(y=df_sorted.value, x=df_sorted.date, title='Bean Price', labels=dict(x='Date', y='USD')).data[0]
# dist.update_layout(width=700, height=700)

fig.add_trace(go.Scatter(dist, line=dict(color='red')), row=1, col=2)
fig.add_trace(hist, row=1, col=2)
fig.add_trace(go.Scatter(price), row=1,col=1)
fig.add_trace(go.Box(x=df_sorted.value), row=2,col=1)
fig.add_trace(go.Box(x=df_sorted['pct_t-1']), row=2,col=2)

fig['layout'].update(height=800, width=1500, title='Returns Descriptive Statistics')

fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_yaxes(title_text="USD", row=1, col=1)

# fig.add_trace(dist, row=1, col=1)
fig.show()